In [1]:
import lib.xmlreader as xml
import lib.utils as ut
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
#import gensim.models.word2vec

In [2]:
train_docs = xml.readXML("../database/TASS/TASS2017/task1-Training.xml",[0,1,2,3])
val_docs   = xml.readXML("../database/TASS/TASS2017/task1-Development.xml",[0,1,2,3])
test_docs  = xml.readXML("../database/TASS/TASS2017/task1-Test.xml",[0,1,2,3])
general_docs = xml.readXML2("../database/TASS/TASS2017/task1-General.xml",[0,1,2,3])

In [3]:
train_tweets = []
train_labels = []
for doc in train_docs:
    # train_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    train_tweets.append(doc.content)
    train_labels.append(doc.polarity)

val_tweets = []
val_labels = []
for doc in val_docs:
    # test_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    val_tweets.append(doc.content)
    val_labels.append(doc.polarity)
    
test_tweets = []
test_labels = []
for doc in test_docs:
    # test_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    test_tweets.append(doc.content)
    test_labels.append(doc.polarity)
    
general_tweets = []
general_labels = []
for doc in general_docs:
    # test_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    general_tweets.append(doc.content)
    general_labels.append(doc.polarity)

In [4]:
len(train_tweets), len(val_tweets), len(test_tweets), len(general_tweets)

(1008, 506, 1899, 7218)

In [57]:
def flattern(A):
    'Source: https://stackoverflow.com/a/17867797/7273299'
    
    'Flattens a list of lists and strings into a list.'
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flattern(i))
        else: rt.append(i)
    return rt

def isInt(v):
    'Source: https://stackoverflow.com/a/9859202/7273299'
    
    'Checks if a string is a number.'
    try:     i = int(v)
    except:  return False
    return True

def char_count(word, chars, lbound=2):
    char_count = [word.count(char) for char in chars]
    return all(i >= lbound for i in char_count)

def replace_lol(repl_str, texts):
    for string, chars in repl_str:
        texts = [[[string, i] if char_count(i, set(chars), 2) else i for i in text.split()] for text in texts]
        texts = np.array([flattern(text) for text in texts])
        texts = np.array([' '.join(text) for text in texts])
    return texts

def preprocess_tweets(tweets):
    """
    twitter specific text processing and shuffle
    """
    # Placeholders for hyperlinks and user references
    tweets = [['hyp_link' if i.startswith('http') 
               else 'user_ref' if i.startswith('@')
               else i for i in tweet.split()] for tweet in tweets]
    tweets = np.array([' '.join(i) for i in tweets])

    # Prefix for Hashtags
    tweets = [[['hash_tag', i] if i.startswith('#') else i for i in tweet.split()] for tweet in tweets]
    tweets = np.array([flattern(tweet) for tweet in tweets])
    tweets = np.array([' '.join(i) for i in tweets])

    # Prefix for integers
    tweets = [[['int_string', i] if isInt(i) else i for i in tweet.split()] for tweet in tweets]
    tweets = np.array([flattern(tweet) for tweet in tweets])
    tweets = np.array([' '.join(i) for i in tweets])

    # Prefix for slang
    tweets = [[['que', 'slang_string'] if i=='q' else ['por', 'slang_string'] if i=='x' else ['de', 'slang_string'] if i=='d' else ['Que', 'slang_string'] if i=='Q' else ['Por', 'slang_string'] if i=='X' else ['De', 'slang_string'] if i=='D' else i for i in tweet.split()] for tweet in tweets]
    tweets = np.array([flattern(tweet) for tweet in tweets])
    tweets = np.array([' '.join(i) for i in tweets])

    # Lol type characters
    repl_str = [('risa_ja','ja'), ('risa_ji','ji'), ('risa_je','je'), ('risa_jo','jo'), ('risa_ju', 'ju'),
               ('risa_ja','aj'), ('risa_ji','ij'), ('risa_ju', 'uj')]

    # Adding prefix to lol type characters
    tweets = replace_lol(repl_str, tweets)

   
    
    return tweets



In [58]:
train_tweets = preprocess_tweets(train_tweets)
val_tweets   = preprocess_tweets(val_tweets)
test_tweets  = preprocess_tweets(test_tweets)
general_tweets  = preprocess_tweets(general_tweets)

In [5]:
col_names = ['label', 'tweet']
df_trn = pd.DataFrame({'tweet':train_tweets, 'label':train_labels},
                      columns=col_names)

df_val = pd.DataFrame({'tweet':val_tweets, 'label':val_labels},
                      columns=col_names)

df_test = pd.DataFrame({'tweet':test_tweets, 'label':test_labels},
                      columns=col_names)
df_general = pd.DataFrame({'tweet':general_tweets, 'label':general_labels},
                      columns=col_names)

In [6]:
df_general.head(20)

,label,tweet
0,3,"Salgo de #VeoTV , que día más largoooooo..."
1,2,@PauladeLasHeras No te libraras de ayudar me/n...
2,1,@marodriguezb Gracias MAR
3,0,"Off pensando en el regalito Sinde, la que se v..."
4,1,Conozco a alguien q es adicto al drama! Ja ja ...
5,3,"RT @FabHddzC: Si amas a alguien, déjalo libre...."
6,1,Toca @crackoviadeTV3 . Grabación dl especial N...
7,3,Hoy asisitiré en Madrid a un seminario sobre l...
8,1,Buen día todos! Lo primero mandar un abrazo gr...
9,1,Desde el escaño. Todo listo para empezar #endi...


In [7]:
df_trn.to_csv('../database/TASS/TASS2017/train.csv', header=False, index=False)
df_val.to_csv('../database/TASS/TASS2017/validation.csv', header=False, index=False)
df_test.to_csv('../database/TASS/TASS2017/test.csv', header=False, index=False)
df_general.to_csv('../database/TASS/TASS2017/general.csv', header=False, index=False)

In [23]:
POSI_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 0]
NEGA_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 1]
NEUT_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 2]
NONE_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 3]

level_train_docs = [POSI_train_docs,NEGA_train_docs,NEUT_train_docs,NONE_train_docs]

fmt = """Positive Sentences = {:d}
       \rNegative Sentences = {:d}
       \rNeutral  Sentences = {:d}
       \rNone Values        = {:d}"""

print(fmt.format(len(POSI_train_docs),
                 len(NEGA_train_docs),
                 len(NEUT_train_docs),
                 len(NONE_train_docs)))

Positive Sentences = 242
Negative Sentences = 231
Neutral  Sentences = 166
None Values        = 361


In [24]:
minSentLvl = min(len(POSI_train_docs),len(NEGA_train_docs),len(NEUT_train_docs),len(NONE_train_docs))

print('Minimum number of sentences per level : ', minSentLvl)

Minimum number of sentences per level :  166


In [25]:
import random

new_train_docs = []
for i in range(len(level_train_docs)):
    level_per = random.sample(level_train_docs[i],len(level_train_docs[i]))
    new_train_docs.append(level_per[:minSentLvl])

In [26]:
print("New size of sentences:\n")
fmt = """Positive Sentences = {:d}
       \rNegative Sentences = {:d}
       \rNeutral  Sentences = {:d}
       \rNone Values        = {:d}"""

print(fmt.format(len(new_train_docs[0]),
                 len(new_train_docs[1]),
                 len(new_train_docs[2]),
                 len(new_train_docs[3])))

New size of sentences:

Positive Sentences = 166
Negative Sentences = 166
Neutral  Sentences = 166
None Values        = 166


In [27]:
flat_train_docs = [item for sublist in new_train_docs for item in sublist]
shuf_train_docs = random.sample(flat_train_docs,len(flat_train_docs))

assert (len(shuf_train_docs) == 4 * minSentLvl)
print("shuf_train_docs size = ", len(shuf_train_docs))

shuf_train_docs size =  664


In [28]:
corpus = []
for doc in shuf_train_docs + val_docs:
    corpus.append(doc.content)

In [29]:
print("Sentences = ", (len(val_docs + shuf_train_docs)))

Sentences =  1164


In [30]:
shuf_train_labels = []
for doc in shuf_train_docs:
    shuf_train_labels.append(doc.polarity)
    
assert (len(shuf_train_labels) == len(shuf_train_docs))

In [31]:
counter = CountVectorizer(tokenizer=ut.tokenizer)

In [32]:
X = counter.fit_transform(corpus)
print(X.shape)

(1164, 4653)


In [33]:
test_docs   = xml.readXMLTest("../database/TASS/TASS2018/task1-Test.xml")

In [34]:
test_tweets = []
for doc in test_docs:
    test_tweets.append(doc.content)

In [35]:
print(len(test_tweets))
# assert (len(test_tweets) == 1899)

1428


In [36]:
sequences = []
for tweet in corpus + test_tweets:
    sentence = []
    for word in ut.tokenizer(tweet):
        sentence.append(word)
    sequences.append(sentence)

In [37]:
assert (len(sequences) == (len(shuf_train_docs) + len(val_docs) + len(test_tweets)))

In [38]:
from collections import Counter

In [39]:
cnt = Counter(word for doc in sequences for word in doc )

In [40]:
cnt['felicidad'], len(cnt)

(11, 8103)

In [41]:
cnt.most_common()[-1]

('activacion', 1)

In [42]:
cnt.most_common()[:10]

[('que', 1458),
 ('de', 1233),
 ('y', 1045),
 ('a', 901),
 ('no', 878),
 ('la', 820),
 ('me', 739),
 ('el', 725),
 ('en', 647),
 ('es', 567)]

In [43]:
import collections

max_vocab = len(cnt)
min_freq = 0
#max_vocab = len(cnt)
#max_vocab = 4000
#min_freq  = 0


itos = [o for o,c in cnt.most_common(max_vocab) if c >= min_freq]

itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

8105

In [44]:
print(len(cnt))

8103


In [45]:
len(stoi)

8105

In [46]:
import pickle

TASS_     = '2018'

In [47]:
pickle.dump(itos, open('../database/ulmfit/tmp/itos_'+TASS_+'.pkl', 'wb'))

In [48]:
itos      = pickle.load(open('../database/ulmfit/tmp/itos_'+TASS_+'.pkl', 'rb'))

In [49]:
data_ids = [[stoi[o] for o in p] for p in sequences]

In [50]:
assert (len(sequences) == len(data_ids))

In [51]:
x_train_ids = np.array(data_ids[:len(shuf_train_labels)])
x_val_ids   = np.array(data_ids[len(shuf_train_labels):len(shuf_train_labels)+len(val_docs)])
x_test_ids  = np.array(data_ids[(len(shuf_train_labels)+len(val_docs)):])

print('Shape of data train tensor:', x_train_ids.shape)
print('Shape of data val  tensor:', x_val_ids.shape)
print('Shape of data test  tensor:', x_test_ids.shape)

Shape of data train tensor: (664,)
Shape of data val  tensor: (500,)
Shape of data test  tensor: (1428,)


In [52]:
SAVE_PATH = '../database/ulmfit/'

np.save(SAVE_PATH + "tmp/train_labels_"+TASS_+".npy",shuf_train_labels)
np.save(SAVE_PATH + "tmp/val_labels_"+TASS_+".npy",val_labels)

In [53]:
np.save(SAVE_PATH + "tmp/train_ids_"+TASS_+".npy",x_train_ids)
np.save(SAVE_PATH + "tmp/val_ids_"+TASS_+".npy",x_val_ids)
np.save(SAVE_PATH + "tmp/test_ids_"+TASS_+".npy",x_test_ids)

In [54]:
sequences[0]

['necesito',
 'que',
 'sea',
 'viernes',
 'para',
 'que',
 'paguen',
 'estoy',
 'en',
 'la',
 'miseria',
 'tota']

In [55]:
itos

['_unk_',
 '_pad_',
 'que',
 'de',
 'y',
 'a',
 'no',
 'la',
 'me',
 'el',
 'en',
 'es',
 'mi',
 'lo',
 'un',
 'se',
 'por',
 'con',
 'te',
 'los',
 'pero',
 'ya',
 'si',
 'una',
 'para',
 'yo',
 'mas',
 'las',
 'como',
 'o',
 'al',
 'todo',
 'tu',
 'solo',
 'esta',
 'cuando',
 'del',
 'quiero',
 'dia',
 'porque',
 'estoy',
 'hoy',
 'mejor',
 'muy',
 'tengo',
 'feliz',
 'mis',
 'bueno',
 'eso',
 'q',
 'su',
 'ser',
 'siempre',
 'bien',
 'le',
 'extra',
 'ahora',
 'este',
 'sin',
 'buen',
 'gracias',
 'ver',
 'ni',
 'todos',
 'asi',
 'hay',
 'tan',
 'hace',
 'vida',
 'os',
 'son',
 'soy',
 'fue',
 'ao',
 'nos',
 'eres',
 'ma',
 'hacer',
 'nada',
 'hasta',
 'ir',
 'mucho',
 'he',
 'dias',
 'buena',
 'cosas',
 'estar',
 'desde',
 'vez',
 'algo',
 'sea',
 'igual',
 'nuevo',
 'ana',
 'tambien',
 'ese',
 'espero',
 'sus',
 'voy',
 'tiempo',
 'peru',
 'tus',
 'creo',
 'semana',
 'sera',
 'era',
 'aun',
 'esa',
 'ha',
 'menos',
 'siento',
 'cada',
 'mal',
 'lindo',
 'tiene',
 'donde',
 'va',
 